<a href="https://colab.research.google.com/github/AnaBelenCarbajal/Thesis/blob/main/shapesTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install osculari

In [141]:
# importing required packages
import osculari

import numpy as np
from matplotlib import pyplot as plt
import torch
import os
from random import choice, randint, uniform

from torchvision.datasets import ImageFolder
from torchvision.io import read_image
from torch.utils.data import Dataset

In [ ]:
# unzip folder (previously uploaded in collab)
!unzip images180.zip -d my_data

In [92]:
# root directory to data
images180 = "my_data/images180/images180/"

In [161]:
# get category labels
labels = os.listdir(images180)

In [ ]:
# get list of dictionaries with respective label and image number for all images
data = []

for label in labels:
  if label == ".DS_Store":
    continue
  folder_path = images180 + label
  shapes = os.listdir(folder_path)
  for shape in shapes:
    category_dict = {'label': label, 'image': shape}
    data.append(category_dict)

data[:10]

In [162]:
# function to get image directory
def image_direct(root, category_dict_item):
  return root + category_dict_item['label'] + "/" + category_dict_item['image']

# get image directories
img_dir = []
for image in data:
  direct = image_direct(images180, image)
  img_dir.append(direct)

img_dir[:10]

['my_data/images180/images180/leopard/41.png',
 'my_data/images180/images180/leopard/22.png',
 'my_data/images180/images180/leopard/89.png',
 'my_data/images180/images180/leopard/46.png',
 'my_data/images180/images180/leopard/31.png',
 'my_data/images180/images180/leopard/69.png',
 'my_data/images180/images180/leopard/98.png',
 'my_data/images180/images180/leopard/1.png',
 'my_data/images180/images180/leopard/39.png',
 'my_data/images180/images180/leopard/24.png']

In [158]:
class ShapeImageDataset (Dataset):
  # return two images and ground truth
  def __init__(self, data, img_dir, labels, transform=None):
    self.img_labels = data
    self.image_dir = img_dir
    self.labels = labels
    self.transform = transform

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):

    #stimulus1
    pick_category1 = choice(self.labels)
    pick_image_numb1 = randint(1, 100)
    image_numb_png1 = str(pick_image_numb1) + 'png'
    directory_ending1 = pick_category1 + "/" + image_numb_png1

    for dir1 in self.image_dir:
      if directory_ending1 in dir1:
        stimuli1 = read_image(dir1)

    #stimulus2
    random_numb = uniform(0,1)
    pick_image_numb2 = randint(1, 100)

    # half of the times fom same category as stimulus1 (gt = "same")
    if random_numb >= 0.5:
      pick_category2 = pick_category1
      image_numb_png2 = str(pick_image_numb1) + 'png'
      directory_ending2 = pick_category2 + "/" + image_numb_png2

      for dir2 in self.image_dir:
        if directory_ending2 in dir2:
          stimuli2 = read_image(dir2)

    # half of the times form another category as stimulus1 (gt = "different")
    else:
       other_categories = self.labels.remove(pick_category1)
       pick_category2 = choice(other_categories)
       image_numb_png2 = str(pick_image_numb1) + 'png'
       directory_ending2 = pick_category2 + "/" + image_numb_png2

       for dir2 in self.image_dir:
         if directory_ending2 in dir2:
           stimuli2 = read_image(dir2)

    # ground truth
    if pick_category1 == pick_category2:
      gt = "same"
    else:
      gt = 'different'

    if self.transform:
      stimuli1 = self.transformation(stimuli1)
      stimuli2 = self.transformation(stimuli2)

    return stimuli1, stimuli2, gt

In [74]:
from torch.utils.data import DataLoader

In [75]:
b = DataLoader(a, 64)